Sentiment analysis 

Trying to use the roberta sentiment prediction: 

https://colab.research.google.com/github/chrsiebert/sentiment-roberta-large-english/blob/main/sentiment_roberta_prediction_example.ipynb

paper: 


In [1]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [2]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)


In [10]:
#load data

trump= pd.read_csv('trump_tweets_languages.csv',lineterminator='\n')

## I used my old csv as I didn't want to compute the language detection again, it takes so long with my laptop.

In [11]:
## filter data frame for english tweets only 
is_english = trump['tweet_lang\r']== 'en\r'

english_tweets= trump[is_english]
print(english_tweets.shape)

(705893, 23)


In [5]:
pred_texts = list(english_tweets['cleantweet'])
## cut the samples down to 100 in order to try 
pred_texts = pred_texts[:100]

In [6]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [7]:
# Run predictions
predictions = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 100
  Batch size = 8


In [8]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [9]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['cleantweet','pred','label','score'])
df.head()

,cleantweet,pred,label,score
0,trump as a student i used to hear for years ...,0,NEGATIVE,0.998256
1,hours since last tweet from trump maybe he...,0,NEGATIVE,0.998891
2,you get a tie and you get a tie trump s ra...,0,NEGATIVE,0.992305
3,clady her minutes were over long time ag...,0,NEGATIVE,0.999265
4,richardmarx glad u got out of the house dick...,1,POSITIVE,0.998635


In [ ]:
## need to join this with our data frame once we computed it for all 

## would like to compare efficientcy& score values of text blob and this, but that would mean we need some manually labled tweets- which I am happy to do for 100 but is that enough? 
